In [3]:

from fddbenchmark import FDDDataset, FDDDataloader, FDDEvaluator
from fdd_defense.attacks import FGSMAttack, PGDAttack, DeepfoolLinfAttack, NoiseAttack
from fdd_defense.models.base import BaseTorchModel
from fdd_defense.models import LinearModel
import torch. nn as nn
from torch.optim import Adam
import torch
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
from fdd_defense.defenders import DistillationDefender, QuantizationDefender, AdversarialTrainingDefender

# Обучаем модель на small_tep

In [4]:
dataset = FDDDataset(name='small_tep')
dataset.df.head()
scaler = StandardScaler()
scaler.fit(dataset.df[dataset.train_mask])
dataset.df[:] = scaler.transform(dataset.df)

Reading data/small_tep/test_mask.csv: 100%|██████████| 153300/153300 [00:00<00:00, 1249476.40it/s]


In [5]:
window_size = 10

fddmodel = LinearModel(
        window_size=window_size,
        step_size=1,
        num_epochs=29,
        batch_size=166,
        lr=0.004209,
    )
fddmodel.fit(dataset)

Creating sequence of samples: 100%|██████████| 105/105 [00:00<00:00, 797.62it/s]


Epochs ...:   0%|          | 0/29 [00:00<?, ?it/s]

Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 1, Loss: 1.3003


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

...

Epoch 27, Loss: 0.9635


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 28, Loss: 0.9625


Steps ...:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 29, Loss: 0.9616


Замеряем метрики

In [6]:
test_loader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=window_size,
    step_size=1,
)
for sample, index, label in test_loader:
    pred = fddmodel.predict(sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

Creating sequence of samples: 100%|██████████| 105/105 [00:00<00:00, 688.93it/s]


FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9720/0.0000
    Fault 02: 0.9810/0.0000
    Fault 03: 0.0013/0.0000
    Fault 04: 0.9865/0.0000
    Fault 05: 0.9835/0.0000
    Fault 06: 0.8568/0.0000
    Fault 07: 0.9992/0.0000
    Fault 08: 0.5612/0.0013
    Fault 09: 0.0000/0.0000
    Fault 10: 0.0053/0.0000
    Fault 11: 0.0013/0.0000
    Fault 12: 0.2671/0.0001
    Fault 13: 0.4010/0.0012
    Fault 14: 0.0000/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0025/0.0000
    Fault 17: 0.9454/0.0000
    Fault 18: 0.8798/0.0001
    Fault 19: 0.0110/0.0000
    Fault 20: 0.8088/0.0000
Detection TPR: 0.5604
Detection FPR: 0.0027
Average Detection Delay (ADD): 112.46
Total Correct Diagnosis Rate (Total CDR): 0.8622

Clustering metrics
-----------------
Adjusted Rand Index (ARI): 0.2454
Normalized Mutual Information (NMI): 0.6346
Unsupervised Clustering Accuracy (ACC): 0.5855


#### Fpr - 0.27% CDR 86%

## Повторяем на reinartz_tep

In [7]:
dataset = FDDDataset(name='reinartz_tep')
dataset.df.head()
scaler = StandardScaler()
scaler.fit(dataset.df[dataset.train_mask])
dataset.df[:] = scaler.transform(dataset.df)

Reading data/reinartz_tep/test_mask.csv: 100%|██████████| 5600000/5600000 [00:05<00:00, 1083265.60it/s]


In [8]:
window_size = 10

fddmodel_M = LinearModel(
        window_size=window_size,
        step_size=1,
        num_epochs=29,
        batch_size=166,
        lr=0.004209,
    )
fddmodel_M.fit(dataset)

Creating sequence of samples: 100%|██████████| 2240/2240 [01:55<00:00, 19.43it/s]


Epochs ...:   0%|          | 0/29 [00:00<?, ?it/s]

Steps ...:   0%|          | 0/26854 [00:00<?, ?it/s]

Epoch 1, Loss: 0.4772


Steps ...:   0%|          | 0/26854 [00:00<?, ?it/s]

...

Epoch 27, Loss: 0.4682


Steps ...:   0%|          | 0/26854 [00:00<?, ?it/s]

Epoch 28, Loss: 0.4682


Steps ...:   0%|          | 0/26854 [00:00<?, ?it/s]

Epoch 29, Loss: 0.4682


In [9]:
test_loader = FDDDataloader(
    dataframe=dataset.df,
    mask=dataset.test_mask,
    labels=dataset.labels,
    window_size=window_size,
    step_size=1,
)
for sample, index, label in test_loader:
    pred = fddmodel_M.predict(sample)
pred = pd.Series(pred, index=index, name='pred')

evaluator = FDDEvaluator(step_size=1)
evaluator.print_metrics(label, pred)

Creating sequence of samples: 100%|██████████| 560/560 [00:49<00:00, 11.34it/s]


FDD metrics
-----------------
TPR/FPR:
    Fault 01: 0.9876/0.0000
    Fault 02: 0.9881/0.0000
    Fault 03: 0.0549/0.0000
    Fault 04: 0.9802/0.0000
    Fault 05: 0.1517/0.0003
    Fault 06: 0.9993/0.0000
    Fault 07: 0.9985/0.0000
    Fault 08: 0.3086/0.0000
    Fault 09: 0.0000/0.0000
    Fault 10: 0.0000/0.0000
    Fault 11: 0.0441/0.0000
    Fault 12: 0.0000/0.0000
    Fault 13: 0.6943/0.0000
    Fault 14: 0.0000/0.0000
    Fault 15: 0.0000/0.0000
    Fault 16: 0.0000/0.0000
    Fault 17: 0.9695/0.0000
    Fault 18: 0.9441/0.0000
    Fault 19: 0.0434/0.0000
    Fault 20: 0.9744/0.0000
    Fault 21: 0.0000/0.0000
    Fault 22: 0.0000/0.0000
    Fault 23: 0.0005/0.0000
    Fault 24: 0.0925/0.0000
    Fault 25: 0.0174/0.0000
    Fault 26: 0.0298/0.0000
    Fault 27: 0.0000/0.0000
    Fault 28: 0.0000/0.0000
Detection TPR: 0.3625
Detection FPR: 0.0003
Average Detection Delay (ADD): 118.11
Total Correct Diagnosis Rate (Total CDR): 0.9143

Clustering metrics
-----------------
Adjusted